## RevLVDT + VC Simulation

In [1]:
import femm
import numpy as np
import cmath
import scipy.optimize as opt
import matplotlib.pyplot as plt

# The package must be initialized with the openfemm command.
femm.openfemm()

# We need to create a new Magnetostatics document to work on.
femm.newdocument(0)

#### Main Parameters

In [2]:
Description_Txt   = "RevLVDT_VC ETpathfinder large IP geometry"
outputfile        = 'RevLVDT_VC_DC_1A_RSpro_40mm_single_layer_1.out'
MIRCoil_Distance  = 0 # offset distance for MIRCoil
MIRCoil_Current   = 1 # Put current on the MIRCoil
SimulationFreq    = 0 # AC frequency = 0: simulate DC situation
NSteps            = 5
StepSize          = 1
MIRCoil_Offset    = -20 # offset distance for MIRCoil

# Define the problem type.  Magnetostatic; Units of mm; Axisymmetric; 
# Precision of 10^(-10) for the linear solver
femm.mi_probdef( SimulationFreq, 'millimeters', 'axi', 1.0e-10)

#### MIRCoil Parameters

In [8]:
# RS pro
MIRCoil_WireType     = "RS pro"
MIRCoil_WireDiam     = 0.2    # diameter of the wire:0.233mm
MIRCoil_WireInsul    = 0.0165
MIRCoil_NrWireLayers = 4
MIRCoil_InRadius     = 10
MIRCoil_Height       = 5.2

#### Magnet Parameters

In [4]:
Magnet_Length    =  40
Magnet_Diameter  = 10
Magnet_Material  = "N40"
Magnet_VertShift =  0 

#### Position calculation for all Coils

In [9]:
# InnerCoil position 
MIRCoil_OutRadius = MIRCoil_InRadius +((MIRCoil_WireDiam + MIRCoil_WireInsul * 2) * MIRCoil_NrWireLayers)
MIRCoil_LowEnd = (MIRCoil_Distance - MIRCoil_Height) / 2 
MIRCoil_UppEnd = MIRCoil_LowEnd + MIRCoil_Height
MIRCoil_NrWind_p_Layer = (MIRCoil_Height) / (MIRCoil_WireDiam + MIRCoil_WireInsul * 2)
MIRCoil_NrWindings = MIRCoil_NrWind_p_Layer * MIRCoil_NrWireLayers
MIRCoil_Circuit = " MIRCoil_Circuit"

print("MIRCoil coil:\n" 
      "MIRCoil inn radius = {:.2f}".format(MIRCoil_InRadius), 
      "MIRCoil out radius = {:.2f}".format(MIRCoil_OutRadius),
      "Lower end = {:.2f}".format(MIRCoil_LowEnd), 
      "Upper end = {:.2f}".format(MIRCoil_UppEnd),
      "#Windings/layer = {:.2f}".format(MIRCoil_NrWind_p_Layer), 
      "Total windings = {:.2f}".format(MIRCoil_NrWindings))
# Total length Calculation
InnCoil_TotalWire = 0
for i in range(0,MIRCoil_NrWireLayers):
    #circ = 2*np.pi*InnCoil_InRadius+i*(InnCoil_WireDiam+InnCoil_WireInsul)
    circ= 2*np.pi*(MIRCoil_InRadius+i*(MIRCoil_WireDiam+MIRCoil_WireInsul * 2))
    InnCoil_TotalWire += circ*MIRCoil_NrWind_p_Layer    
print("Total length of wire (mm):", InnCoil_TotalWire)

# Magnet Position
Magnet_UppEnd = Magnet_Length / 2 + Magnet_VertShift
Magnet_LowEnd = -Magnet_Length / 2 + Magnet_VertShift
Magnet_Radius = Magnet_Diameter / 2

print("Voice coil magnet:\n"
      "Magnet Radius = {:.2f}".format(Magnet_Radius),
      "Upper end = {:.2f}".format(Magnet_UppEnd),
      "Lower end = {:.2f}".format(Magnet_LowEnd))

MIRCoil coil:
MIRCoil inn radius = 10.00 MIRCoil out radius = 10.93 Lower end = -2.60 Upper end = 2.60 #Windings/layer = 22.32 Total windings = 89.27
Total length of wire (mm): 5805.059175117711
Voice coil magnet:
Magnet Radius = 5.00 Upper end = 20.00 Lower end = -20.00


#### Build Geometry

In [6]:
# MIRCoil Structure
femm.mi_drawrectangle(MIRCoil_InRadius, MIRCoil_UppEnd, MIRCoil_OutRadius, MIRCoil_LowEnd)
femm.mi_addcircprop(MIRCoil_Circuit, MIRCoil_Current, 1)
## Select material

# select from material library
# femm.mi_getmaterial(MIRCoil_WireType)
# RS pro
femm.mi_addmaterial('RS pro', 1, 1, 0, 0, 58, 0, 0, 1, 3, 0, 0, 1, 0.2)
femm.mi_clearselected()
femm.mi_selectrectangle(MIRCoil_InRadius,MIRCoil_UppEnd, MIRCoil_OutRadius, MIRCoil_LowEnd, 4)
femm.mi_setgroup(1)
femm.mi_clearselected()
femm.mi_addblocklabel(MIRCoil_InRadius + MIRCoil_WireDiam,MIRCoil_LowEnd + (MIRCoil_Height/2))
femm.mi_selectlabel(  MIRCoil_InRadius + MIRCoil_WireDiam,MIRCoil_LowEnd + (MIRCoil_Height/2))
femm.mi_setblockprop(MIRCoil_WireType, 1, 0, MIRCoil_Circuit, 0, 1 ,MIRCoil_NrWindings)
femm.mi_clearselected()


# Magnet Structure
femm.mi_drawrectangle(0, Magnet_UppEnd, Magnet_Radius,Magnet_LowEnd)
femm.mi_getmaterial(Magnet_Material)
femm.mi_clearselected()
femm.mi_selectrectangle(0, Magnet_UppEnd, Magnet_Radius,Magnet_LowEnd, 4)
femm.mi_setgroup(2)
femm.mi_clearselected()
femm.mi_addblocklabel(Magnet_Radius*0.5, Magnet_LowEnd+(Magnet_Length*0.5))
femm.mi_selectlabel(  Magnet_Radius*0.5, Magnet_LowEnd+(Magnet_Length*0.5))
femm.mi_setblockprop(Magnet_Material, 0, 0.1, "", 90, 2, 0)
femm.mi_clearselected()

# AirSurrounding Structure
AirSpaceRadius_1 = 100
AirSpaceRadius_2 = 300
BC_Name = "Outside"
BC_Group = 10
# Airspace1
femm.mi_drawline(0, AirSpaceRadius_1, 0, -AirSpaceRadius_1)
femm.mi_drawarc(0, -AirSpaceRadius_1, 0, AirSpaceRadius_1, 180, 2)
femm.mi_getmaterial("Air")       
femm.mi_clearselected()
femm.mi_addblocklabel(AirSpaceRadius_1/4, AirSpaceRadius_1/2)
femm.mi_selectlabel(  AirSpaceRadius_1/4, AirSpaceRadius_1/2)
femm.mi_setblockprop("Air", 0, 0.5, '', 0, 0, 0)
femm.mi_clearselected()
# Airspace2              
femm.mi_drawline(0, AirSpaceRadius_2, 0, -AirSpaceRadius_2)
femm.mi_drawarc(0, -AirSpaceRadius_2, 0, AirSpaceRadius_2, 180, 2)
femm.mi_getmaterial("Air")  
femm.mi_clearselected()
femm.mi_addblocklabel(AirSpaceRadius_2/2, AirSpaceRadius_2/1.2)
femm.mi_selectlabel(  AirSpaceRadius_2/2, AirSpaceRadius_2/1.2)
femm.mi_setblockprop("Air", 1, 0, '', 0, 0, 0)
femm.mi_clearselected()
# Boundary properties
femm.mi_addboundprop(BC_Name, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
femm.mi_clearselected()
femm.mi_selectarcsegment(0, AirSpaceRadius_2)
femm.mi_setarcsegmentprop(2, BC_Name, 0, BC_Group)               
femm.mi_clearselected()


#### Start Simulation and change the inner coil position

Output variables

In [ ]:
MIRCoil_Forces = np.zeros(NSteps+1)
Magnet_Forces = np.zeros(NSteps+1)
MIRCoil_Positions = np.zeros(NSteps+1)
MetaData = np.zeros(NSteps+1)

Initial Offset position

In [ ]:
femm.mi_selectgroup(1)
femm.mi_movetranslate(0, MIRCoil_Offset)
femm.mi_clearselected()

Simulation Loop

In [ ]:
for i in range(0,NSteps+1):

    print(MIRCoil_Offset + StepSize*i)
    MIRCoil_Positions[i] = MIRCoil_Offset + StepSize*i

    # Now, the finished input geometry can be displayed.
    #femm.mi_zoomnatural()
    femm.mi_zoom(-2,-50,50,50)
    femm.mi_refreshview()

    # We have to give the geometry a name before we can analyze it.
    femm.mi_saveas('RevLVDT_VC_MIRCoil.fem')

    # Now,analyze the problem and load the solution when the analysis is finished
    femm.mi_analyze()
    femm.mi_loadsolution()

    if NSteps == 0:
        # Show Density Plot:
        femm.mo_showdensityplot(1, 0, 0.0001, 1.0E-9, "bmag")
                #--legend,	(0=hide, 1=show)
                #--gscale,	(0=color, 1=greyscale)
                #--upper_B,	(upperlimit for display)
                #--lower_B,	(lowerlimit for display)
                #--type		("bmag", "breal", "bimag" FluxDensity)
                #--			("hmag", "hreal", "himag" FieldIntensity)
                #--			("jmag", "jreal", "jimag" CurrentDensity)
        femm.mo_zoom(-2,-50,50,50)
        femm.mo_refreshview()

    # Retrieve the Force on the Upper-OutCoil
    femm.mo_groupselectblock(1)
    MIRCoil_Force19 = femm.mo_blockintegral(19)
    femm.mo_clearblock()

    # Retrieve the Force on the magnet
    femm.mo_groupselectblock(2)
    Magn_Force19 = femm.mo_blockintegral(19)
    femm.mo_clearblock()

    print("MIRCoil force = ", MIRCoil_Force19, "Magnet force = ", Magn_Force19)
    MIRCoil_Forces[i] = MIRCoil_Force19
    Magnet_Forces[i] = Magn_Force19

    # Translate inner coil to different distance
    femm.mi_selectgroup(1)
    femm.mi_movetranslate(0, StepSize)
    femm.mi_clearselected()

In [ ]:
print(MIRCoil_Positions)
print(MIRCoil_Forces)
print(Magnet_Forces)

In [ ]:
if NSteps > 2:
   MetaData[0] = NSteps
   MetaData[1] = StepSize
   MetaData[2] = MIRCoil_Current
   np.savetxt(outputfile, (MIRCoil_Positions, MIRCoil_Forces, Magnet_Forces, MetaData))

In [ ]:
plt.style.use(['science','grid','notebook'])

plt.plot(MIRCoil_Positions, MIRCoil_Forces, 'o-')
plt.ylabel('MIR Coil Force [N]')
plt.xlabel('MIRCoil Position [mm]')
plt.show()

plt.plot(MIRCoil_Positions, Magnet_Forces, 'o-')
plt.ylabel('Magnet Force [N]')
plt.xlabel('MIRCoil Position [mm]')
plt.show()

plt.plot(MIRCoil_Positions, Magnet_Forces/max(Magnet_Forces)*100, 'o-')
plt.ylabel('Magnet Forces/max force [%]')
plt.xlabel('Inner Coil Position [mm]')
plt.show()

In [ ]:
def polyfunc(x, a, b, c):
    return a*x**2 + b*x + c

Norm_Magnet_Forces = Magnet_Forces/MIRCoil_Current 
optimizedParameters, pcov = opt.curve_fit(polyfunc, MIRCoil_Positions, Norm_Magnet_Forces)
print("Fitted parameters of function:", optimizedParameters)
fitted_Norm_Magnet_Forces = polyfunc(MIRCoil_Positions, *optimizedParameters)

plt.plot(MIRCoil_Positions, Norm_Magnet_Forces, label="simulation")
plt.plot(MIRCoil_Positions, fitted_Norm_Magnet_Forces, '--', label="poly2 fit")
plt.ylabel('Normalised Magnet Force [N/A]')
plt.xlabel('Inner Coil Position [mm]')
plt.legend()
plt.show()

plt.plot(MIRCoil_Positions, Norm_Magnet_Forces - fitted_Norm_Magnet_Forces)
plt.ylabel('Fit error [N/A]')
plt.xlabel('Inner Coil Position [mm]')
plt.show()

plt.plot(MIRCoil_Positions, abs(Norm_Magnet_Forces - fitted_Norm_Magnet_Forces)/abs(Norm_Magnet_Forces)*100)
plt.ylabel('Normalised Fit error [%]')
plt.xlabel('Inner Coil Position [mm]')
plt.ylim(0.0,20)
plt.show()

In [ ]:
femm.closefemm()